In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import collections as col
from itertools import combinations


In [2]:
#Figure out a way to get these unique paths. 
c=0
# Check if cell `(x, y)` is valid or not, this just checks if it lies within the maze or not.
def isValidCell(x, y,maze):
    return not (x < 0 or y < 0 or x >= N or y >= N)
 
def grid(n,prob):
    g = np.random.choice([1,0], (n*n - 2), p=[(1-prob),prob])
    g = np.concatenate(([1] , g , [1]))
    gr = np.reshape(g , (n,n))
    return gr

#grid, arrays of all arrangements of a given number of mines. Size=n
def gridarray(size,mines):
    for positions in combinations(range(size), mines):
        p = [1] * (size)

        for i in positions:
            p[i] = 0

        yield p

#it reshapes all arrays from p to give matrices of all cofigurations for a particular numebr of mines
def grids(n,m):
    gridlist = []
    g = list(gridarray(n*n -2 , m))
    for gr in g:
        gr = np.concatenate(([1],gr,[1]))
        gr = np.reshape(gr, (n,n))
        gridlist.append(gr)
        
    return gridlist



 
def countPaths(maze, x, y, visited, count,templist,dataframe):
    global count_mine, count_block
    
    # if destination (bottom-rightmost cell) is found,
    # increment the path count
    if x == N - 1 and y == N - 1:
        #templist.append([N-1,N-1])
        #duplicate templist here
        dataframe.loc[len(dataframe.index)] = [list(templist),'s']
        return count + 1
    
 
    # mark the current cell as visited
    visited[x][y] = True
    #print("appended ",x,y)
    #append element here, which can later be removed if it does not satisfy below if statements, we can remove element at backtracking stage.
    templist.append([x,y])
 
    # if the current cell is a valid and open cell
    if isValidCell(x, y,maze) and maze[x][y] == 1:
 
        # go down `(x, y)` ——> `(x + 1, y)`
        if x + 1 < N and not visited[x + 1][y]:
            count = countPaths(maze, x + 1, y, visited, count,templist,dataframe)
 
        # go up `(x, y)` ——> `(x - 1, y)`
        if x - 1 >= 0 and not visited[x - 1][y]:
            count = countPaths(maze, x - 1, y, visited, count,templist,dataframe)

        # go right `(x, y)` ——> `(x, y + 1)`
        if y + 1 < N and not visited[x][y + 1]:
            count = countPaths(maze, x, y + 1, visited, count,templist,dataframe)
 
        # go left `(x, y)` ——> `(x, y - 1)`
        if y - 1 >= 0 and not visited[x][y - 1]:
            count = countPaths(maze, x, y - 1, visited, count,templist,dataframe)
            
    #paths that end in mines,d
    if isValidCell(x, y, maze) and maze[x][y]==0:
        dataframe.loc[len(dataframe.index)] = [list(templist),'d']
        count_mine+=1
        
    #paths blocked, l  
    if isValidCell(x, y, maze) and maze[x][y]==1 and (x,y)!=(N-1,N-1):
        if ((not isValidCell(x+1, y, maze) or visited[x+1][y]) and (x+1,y)!=(N-1,N-1)) and ((not isValidCell(x-1, y, maze) or visited[x-1][y])and (x-1,y)!=(N-1,N-1)) and ((not isValidCell(x, y+1, maze) or visited[x][y+1])and (x,y+1)!=(N-1,N-1)) and ((not isValidCell(x, y-1, maze) or visited[x][y-1])and (x,y-1)!=(N-1,N-1)):
            dataframe.loc[len(dataframe.index)] = [list(templist),'l']
            count_block+=1
                
        
     
    # backtrack from the current cell and remove it from the current path
    visited[x][y] = False
    templist.remove([x,y])
    #print("removed", x,y)
    return count
 
 

if __name__ == '__main__':
 
    #sdataframe is one that holds the list of paths and a label that says if the path was a success( for sdataframe all are "s".)
    pd.set_option("display.max_colwidth", -1)
    data= {'path list':[[0,0,0,0]],
           'path label':['s'] }
    dataframe=pd.DataFrame(data)
    n=6
    prob=0.3
    maze = grid(n,prob)
   
    
    print(maze)
 
    N = len(maze)
 
    # stores number of unique paths from source to destination
    count = 0
    count_mine = 0
    count_block = 0
    templist=[]
    # 2D matrix to keep track of cells involved in the current path
    visited = [[False for x in range(N)] for y in range(N)]
 
    # start from source cell `(0, 0)`
    count= countPaths(maze, 0, 0, visited, count,templist,dataframe)
    
    dataframe=dataframe.drop([0]).reset_index(drop=True)
    
    print("The total number of safe paths, s= ", count)
    print("The total number of paths that hit mines, d= ", count_mine)
    print("The total number of blocked paths, l= ", count_block)
    print("The total number of paths s+d+l = ", count + count_mine + count_block)
    print(dataframe)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


[[1 1 1 0 1 1]
 [1 1 1 1 1 1]
 [0 1 1 0 1 1]
 [1 1 0 0 0 1]
 [0 1 1 1 1 1]
 [1 1 0 1 0 1]]
The total number of safe paths, s=  25
The total number of paths that hit mines, d=  616
The total number of blocked paths, l=  19
The total number of paths s+d+l =  660
                                                                    path list  \
0    [[0, 0], [1, 0], [2, 0]]                                                   
1    [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [5, 2]]           
2    [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [5, 0], [4, 0]]   
3    [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1], [4, 1], [4, 2], [5, 2]]           
4    [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1], [4, 1], [4, 2], [3, 2]]           
..                                                                ...           
655  [[0, 0], [0, 1], [0, 2], [1, 2], [1, 1], [2, 1], [2, 2], [3, 2]]           
656  [[0, 0], [0, 1], [0, 2], [1, 2], [1, 1], [2, 1], [2, 2], [2, 3]]           
657  [[0, 